# Rock Paper Scissors - Anti-RotN

Description from: http://webdocs.cs.ualberta.ca/~darse/rsb-results2.html


Dummy Bot: Anti-rotn
```
  Length (in C semi-colons):   40
  Type of history analysis:    consecutive pairs of opponent actions
  Level of meta-reasoning:     one
  theoretical exploitability:  40 out of 1000
```
The Anti-rotn bot is the only dummy bot that actually analyzed the opponent's moves and tried to win. It has a minimal use of history, counting only the "rotations" in the opponent's move sequence. If the opponent repeated an action, went up by one, or up by two, it was counted as a 0, +1, or -1 rotation, respectively. This means that it treats the actual choice of rock, paper, or scissors as irrelevant, and considers only the difference between each pair of moves. This enables it to pick up a particular pattern faster than if it "split the signal" into three different categories (for example by treating "rp", "ps", and "sr" as three distinct patterns).

The "Anti-rotten-bot" looks for both maximum and minimum occurrences in the distribution, which many of the entries failed to do. For example, if a prediction algorithm believes the probability distribution for the opponent's next action is 20-39-41 (20% rock, 39% paper, 41% scissors), several programs counter only the maximum likelyhood, and would play rock to beat the expected scissors. However, there is a much stronger opponent tendency toward "not rock", and playing scissors would have a much higher expectation.

In general, the expected value of each action "x" is simply:

```
EV[x] = Pr[x+2] - Pr[x+1]
```
where "+" is understood to be modulo 3. So in the 20-39-41 example,

```
EV[rock] = +2%,   EV[paper] = -21%,   and EV[scissors] = +19%.
```

The other feature built into the Anti-rotn bot is a simple fail-safe, set at 4% of the total match length. This single line of code:

```
if ( score < -trials/25 ) { return(random() % 3); }
```
would have saved many entries thousands of points against stronger opposition. The same approach would not prevent a match loss, however. Even though a -40 result is less than the noise level for a single match (+/- 50.6), many independent tournaments can be run to reduce the variance to a small amount, making the -40 expectation decisive.

Some programs, such as Biopic, Robertot, and Boom, only deviate from random when there appears to be a statistically significant weakness to exploit. This enables them to draw against very strong opposition, who do not exhibit such patterns. Interestingly, Iocaine Powder and Phasenbott are still able to defeat these programs by a small margin, presumably by behaving predictably enough to induce a pattern in their opponent's replies, which they can then exploit for a net profit!

In [ ]:
%%writefile anti_rotn.py

import random

history    = []
rotn_stats = [ 0, 0, 0 ]

# observation   =  {'step': 1, 'lastOpponentAction': 1}
# configuration =  {'episodeSteps': 10, 'agentTimeout': 60, 'actTimeout': 1, 'runTimeout': 1200, 'isProduction': False, 'signs': 3}
def anti_rotn(observation, configuration, warmup=25):    
    global history
    global rotn_stats
    
    if observation.step > 0:
        history.append( observation.lastOpponentAction )

    if len(history) >= 2:
        rotn = (history[-1] - history[-2]) % configuration.signs
        rotn_stats[ rotn ] += 1
        
    if observation.step < warmup:
        action = random.randint(0, configuration.signs-1)
    else:
        ev = list({
            0: rotn_stats[2] - rotn_stats[1], 
            1: rotn_stats[0] - rotn_stats[2], 
            2: rotn_stats[1] - rotn_stats[0], 
        }.values())
        offset = ev.index(max(ev))
        action = (offset + observation.lastOpponentAction) % configuration.signs
        print(f'rotn_stats = {rotn_stats} | ev = {ev} | action = {action}')
        
    return int(action)


In [ ]:
%run -i 'anti_rotn.py'

In [ ]:
def random_agent(observation, configuration):
    return random.randint(0, configuration.signs-1)

def rock_agent(observation, configuration):
    return 0

def paper_agent(observation, configuration):
    return 1

def scissors_agent(observation, configuration):
    return 2

In [ ]:
from kaggle_environments import make

env = make("rps", configuration={"episodeSteps": 100}, debug=True)
env.run(["anti_rotn.py", paper_agent])
env.render(mode="ipython", width=600, height=600)

# Further Reading

This notebook is part of a series exploring Rock Paper Scissors:

Predetermined
- [PI Bot](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-pi-bot)
- [Anti-PI Bot](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-anti-pi-bot)
- [Anti-Anti-PI Bot](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-anti-anti-pi-bot)
- [De Bruijn Sequence](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-de-bruijn-sequence)

RNG
- [Random Agent](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-random-agent)
- [Random Seed Search](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-random-seed-search)
- [RNG Statistics](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-rng-statistics)

Opponent Response
- [Anti-Rotn](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-anti-rotn)
- [Sequential Strategies](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-sequential-strategies)

Statistical 
- [Weighted Random Agent](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-weighted-random-agent)
- [Statistical Prediction](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-statistical-prediction)
- [Anti-Rotn Weighted Random](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-anti-rotn-weighted-random)

Memory Patterns
- [Naive Bayes](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-naive-bayes)
- [Memory Patterns](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-memory-patterns)

Decision Tree
- [XGBoost](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-xgboost)
- [Multi Stage Decision Tree](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-multi-stage-decision-tree)
- [Decision Tree Ensemble](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-decision-tree-ensemble)

Ensemble
- [Multi Armed Stats Bandit](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-multi-armed-stats-bandit)

RoShamBo Competition Winners
- [Iocaine Powder](https://www.kaggle.com/jamesmcguigan/rps-roshambo-comp-iocaine-powder)
- [Greenberg](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-greenberg)